# Feature Importance
### 1) data load 및 train ~ test로 split
### 2) Model
### 3) (Feature 별로 reverse해서) 예측해서 loss값 구하기. -- 괄호는 이번 프로젝트를 위한 과정.
### 4) Display LSTM Feature Importance
### 5) Visualization (Graph)



In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings(action = 'ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# 1) data load 및 split
import random

# data seed random 설정
seed_num = 42
random.seed(seed_num)

x = np.load('C:/Users/kelly/Downloads/DSML_data/x_(7727,10,4068).npy')
y = np.load('C:/Users/kelly/Downloads/DSML_data/y_(7727,1).npy')

# 랜덤으로 700개 추출
# random seed에 따른 shuffle

idx = list(range(len(x)))
random.shuffle(idx)
idx = idx[:700]

i = round(len(idx)*0.8)

X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((560, 10, 4068), (560,), (140, 10, 4068), (140,))

In [13]:
type(X_train)

numpy.ndarray

In [14]:
# 2) Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping

# ----------------------
seed_num = 42
# ----------------------
tf.random.set_seed(seed_num)

lstm2 = Sequential()
lstm2.add(InputLayer(input_shape = (X_train.shape[1], X_train.shape[2])))
lstm2.add(LSTM(units=128, activation = 'hard_sigmoid', return_sequences = True))
lstm2.add(LSTM(units=64, activation = 'hard_sigmoid', return_sequences = True))
lstm2.add(Dropout(0.2))

lstm2.add(LSTM(units=64, activation = 'hard_sigmoid', return_sequences = True))
lstm2.add(LSTM(units=32, activation = 'hard_sigmoid', return_sequences = False))
lstm2.add(Dropout(0.2))

lstm2.add(Dense(units=1, activation = 'sigmoid'))

early_stop = EarlyStopping(monitor='val_acc', patience=30, verbose=1, restore_best_weights = True)
# patience : 최고 성능 나온 이후에 학습 얼마나 더 해볼지 (뒤에 또 나올 수 있으니까.)
# verbose : 0,1,2 중 선택, 훈련 진행과정 얼마나 자세히 표시할지 선택
# restore_best_weights : 학습 중에 가장 좋았던 weights 저장할 지 여부
# https://tykimos.github.io/2017/07/09/Early_Stopping/


lstm2.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics = ['acc'])
lstm2.fit(X_train, y_train, validation_split = 0.25, batch_size = 128, epochs = 100)
# validation_split : x,y train에서 일정 비율 분리해서 검증으로 사용. validation_data param 대신 사용.
# batch_size : 훈련 모델에 넣을 덩어리 크기
# epochs : 훈련 횟수
# https://roboreport.co.kr/%EB%94%A5%EB%9F%AC%EB%8B%9Dlstm%EC%9C%BC%EB%A1%9C-%EC%95%84%ED%8C%8C%ED%8A%B8-%EC%A7%80%EC%88%98-%EC%98%88%EC%B8%A1%ED%95%98%EA%B8%B0-2-lstm-%EC%8B%A4%ED%97%98%ED%95%98%EA%B8%B0/

Epoch 1/100
4/4 [==============================] - 88s 2s/step - loss: 0.6831 - acc: 0.5927 - val_loss: 0.6637 - val_acc: 0.6214
Epoch 2/100
4/4 [==============================] - 1s 325ms/step - loss: 0.6677 - acc: 0.5963 - val_loss: 0.6681 - val_acc: 0.6214
Epoch 3/100
4/4 [==============================] - 1s 291ms/step - loss: 0.6931 - acc: 0.5565 - val_loss: 0.6706 - val_acc: 0.6214
Epoch 4/100
4/4 [==============================] - 1s 328ms/step - loss: 0.6910 - acc: 0.5236 - val_loss: 0.6683 - val_acc: 0.6214
Epoch 5/100
4/4 [==============================] - 2s 411ms/step - loss: 0.6840 - acc: 0.5601 - val_loss: 0.6655 - val_acc: 0.6214
Epoch 6/100
4/4 [==============================] - 2s 436ms/step - loss: 0.6887 - acc: 0.5877 - val_loss: 0.6636 - val_acc: 0.6214
Epoch 7/100
4/4 [==============================] - 2s 442ms/step - loss: 0.6815 - acc: 0.5900 - val_loss: 0.6632 - val_acc: 0.6214
Epoch 8/100
4/4 [==============================] - 1s 342ms/step - loss: 0.6814 - acc

In [18]:
np.abs(X_test-1)


array([[[0., 1., 0., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        ...,
        [0., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[0., 1., 0., ..., 1., 1., 1.],
        [0., 1., 0., ..., 1., 1., 1.],
        [0., 1., 0., ..., 1., 1., 1.],
        ...,
        [0., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[0., 1., 0., ..., 1., 1., 1.],
        [0., 1., 0., ..., 1., 1., 1.],
        [0., 1., 0., ..., 1., 1., 1.],
        ...,
        [0., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1., 1.],
        [0., 1., 1., ..., 1., 1.

In [20]:
# 3) feature 별로 reverse해서 예측해서 loss값 구하기.
# cross_entropy의 방법으로는 binary_cross_entropy사용

# feaures (itemid가 피쳐임.)
from tqdm import tqdm

a = pd.read_csv('C:/Users/kelly/Downloads/DSML_data/total_data_7727.csv')
features = list(a['ITEMID'].sort_values().unique()) #4028개의 itemid존재.

results = []
preds = lstm2.predict(X_test) # baseline 설정하기 위해 먼저 preds


# COMPUTE BASELINE
# loss 값 구하기
from tensorflow.keras.losses import BinaryCrossentropy 
# Binary Cross Entropy: loss값의 한 종류
bce = BinaryCrossentropy()
baseline_bce = bce(y_test, preds).numpy() # true, pred를 이용해 loss값 구한게 baseline
results.append({'feature':'BASELINE', 'baseline_bce':baseline_bce})

for k in tqdm(range(len(features))): # feature별 for문. (feature는 shuffle을 해도 되고, reverse(반대로 하는거)를 해도 됨.)

    save_col = X_test[:,:,k].copy() # 초기화

    # Reverse All Feature k
    ## 방법 1 간단, 코드도 짧...
    X_test[:,:,k] = np.abs(X_test[:,:,k]-1)

    ## 방법 2 코드 긺..
    # X_test[:,:,k] = np.where(X_test[:,:,k] == 1, 2, X_test[:,:,k]) # 1이면 2로 바꾸고 아니면 그대로 냅두라는 의미
    # X_test[:,:,k] = np.where(X_test[:,:,k] == 0, 1, X_test[:,:,k])
    # X_test[:,:,k] = np.where(X_test[:,:,k] == 2, 1, X_test[:,:,k])

    
    # Compute BCE with feature k reversed
    pred2 = lstm2.predict(X_test) # diff (baseline_bce와 bce간 차이)위해서 각각의 bce 또 구하는 것.
    loss_bce = bce(y_test, pred2).numpy() # 넘파이로 안하면 그냥 객체<tf.Tensor: shape=(), dtype=float32, numpy=1.4845252>로 나옴.

    results.append({'feature':features[k], 'bce':loss_bce})
    X_test[:,:,k] = save_col #리셋

100%|██████████| 4068/4068 [23:33<00:00,  2.88it/s]


In [21]:
# 4) Display LSTM feature importance
df = pd.DataFrame(results)
df1 = df.copy()
df1['baseline_bce'] = float(df['baseline_bce'].dropna().unique())
df1['diff'] = df1['bce'] - df1['baseline_bce']
df1 = df1.sort_values(by = 'diff')
df1

,feature,baseline_bce,bce,diff
2196,409606211,2.353497,2.196926,-0.156570
1010,54858224,2.353497,2.198983,-0.154514
119,51002,2.353497,2.201319,-0.152178
3724,61553015311,2.353497,2.202256,-0.151240
2736,10019016312,2.353497,2.202829,-0.150668
...,...,...,...,...
3968,66591018442,2.353497,2.656166,0.302669
5,50806,2.353497,2.658753,0.305256
792,29321121,2.353497,2.663199,0.309703
2217,409739172,2.353497,2.675035,0.321538
